Playing around to check various methods. 

In [39]:
from solders.pubkey import Pubkey
from solana.rpc.api import Client

# STEP 1: Connect to Solana network
solana_client = Client("https://api.mainnet-beta.solana.com")

try:
    token_pubkey = Pubkey.from_string("C7heQqfNzdMbUFQwcHkL9FvdwsFsDRBnfwZDDyWYCLTZ")
except:
    print ({"error": "Invalid token pubkey plm"})
    exit()

print ('CA is', token_pubkey)

# Account info
mint_account = solana_client.get_account_info(token_pubkey)
# print (mint_account)
if not mint_account.value:
    print ({"error": "Invalid mint account"})
    exit()
else:
    print ({"mint_account": mint_account.value})
    print ({"mint_account_data": mint_account.value.data})

mint_data = mint_account.value
data_bytes = mint_account.value.data
print ({"data_bytes": data_bytes})
print ({"data_bytes length": len(data_bytes)})

mint_authority_enabled = data_bytes[0] != 0
mint_authority = None
if mint_authority_enabled and len(data_bytes) >= 36:
    mint_authority = str(Pubkey.from_bytes(data_bytes[4:36]))
print ({"mint_authority": mint_authority})

freeze_authority_enabled = False
freeze_authority = None
if len(data_bytes) >= 82 and data_bytes[46] != 0:
    freeze_authority_enabled = True
    try:
        freeze_authority = str(Pubkey.from_bytes(data_bytes[47:79]))
    except Exception as e:
        print(f"Error parsing freeze authority: {e}")


print ({"freeze_authority_enabled?": freeze_authority_enabled})
print ({"freeze_authority": freeze_authority})

program_id = mint_data.owner
program_info = solana_client.get_account_info(program_id)
is_upgradable = program_info.value.owner == Pubkey.from_string("BPFLoaderUpgradeab1e11111111111111111111111")
print ({"is_upgradable": is_upgradable})

CA is C7heQqfNzdMbUFQwcHkL9FvdwsFsDRBnfwZDDyWYCLTZ
{'mint_account': Account(
    Account {
        lamports: 12267743290,
        data.len: 82,
        owner: TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA,
        executable: false,
        rent_epoch: 18446744073709551615,
        data: 0000000006c5c1ce638d2567d26468b05eb951d1a28dcc6e123482b5c675149770e62bf2cbf7355a768d03000601000000000000000000000000000000000000,
    },
)}
{'mint_account_data': b'\x00\x00\x00\x00\x06\xc5\xc1\xcec\x8d%g\xd2dh\xb0^\xb9Q\xd1\xa2\x8d\xccn\x124\x82\xb5\xc6u\x14\x97p\xe6+\xf2\xcb\xf75Zv\x8d\x03\x00\x06\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'}
{'data_bytes': b'\x00\x00\x00\x00\x06\xc5\xc1\xcec\x8d%g\xd2dh\xb0^\xb9Q\xd1\xa2\x8d\xccn\x124\x82\xb5\xc6u\x14\x97p\xe6+\xf2\xcb\xf75Zv\x8d\x03\x00\x06\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

Claude Suggested Code (reiterated)

In [18]:
from solders.pubkey import Pubkey
from solana.rpc.api import Client
import base58

def check_solana_token_security(token_address: str):
    """Analyzes a Solana token's smart contract for security risks"""
    
    # STEP 1: Connect to Solana network
    solana_client = Client("https://api.mainnet-beta.solana.com")
    
    # STEP 2: Convert string address to Pubkey object
    try:
        token_pubkey = Pubkey.from_string(token_address)
    except:
        return {"error": "Invalid token address"}
    
    # STEP 3: Get token's mint account info
    mint_account = solana_client.get_account_info(token_pubkey)
    if not mint_account.value:
        return {"error": "Token account not found"}
    
    mint_data = mint_account.value
    # Remove base58 decoding - data is already binary
    data_bytes = mint_data.data
    
    # STEP 4: Check Mint/Freeze Authorities (critical!)
    # SPL Token data structure offsets:
    # https://spl.solana.com/token#account-structures
    mint_authority_enabled = data_bytes[0] != 0
    mint_authority = None
    if mint_authority_enabled and len(data_bytes) >= 36:
        mint_authority = str(Pubkey.from_bytes(data_bytes[4:36]))
    
    freeze_authority_enabled = False
    freeze_authority = None
    if len(data_bytes) >= 65 and data_bytes[64] != 0:
        freeze_authority_enabled = True
        if len(data_bytes) >= 100:
            freeze_authority = str(Pubkey.from_bytes(data_bytes[68:100]))
    
    # STEP 5: Check if program is upgradable
    program_id = mint_data.owner
    program_info = solana_client.get_account_info(program_id)
    is_upgradable = program_info.value.owner != Pubkey.from_string("BPFLoaderUpgradeab1e11111111111111111111111")
    
    return {
        "mint_address": token_address,
        "owner": str(mint_data.owner),
        "mint_authority_enabled": mint_authority_enabled,
        "mint_authority": mint_authority,
        "freeze_authority_enabled": freeze_authority_enabled,
        "freeze_authority": freeze_authority,
        "is_upgradable_program": is_upgradable
    }

# Example usage
token_address = "FUAfBo2jgks6gB4Z4LfZkqSZgzNucisEHqnNebaRxM1P"
security_report = check_solana_token_security(token_address)
print(security_report)

{'mint_address': 'FUAfBo2jgks6gB4Z4LfZkqSZgzNucisEHqnNebaRxM1P', 'owner': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA', 'mint_authority_enabled': False, 'mint_authority': None, 'freeze_authority_enabled': True, 'freeze_authority': None, 'is_upgradable_program': True}


GPT4 

The output 'freeze_authority_enabled': True, 'freeze_authority': None indicates that the freeze authority is enabled, but the actual public key of the freeze authority could not be determined. This is indeed a red flag because it suggests that there might be an issue with the data structure or the way the data is being parsed.

Let's add some additional checks and debug information to ensure we are correctly parsing the freeze authority:

In [ ]:
from solders.pubkey import Pubkey
from solana.rpc.api import Client

def check_solana_token_security(token_address: str):
    """Analyzes a Solana token's smart contract for security risks"""
    
    # STEP 1: Connect to Solana network
    solana_client = Client("https://api.mainnet-beta.solana.com")
    
    # STEP 2: Convert string address to Pubkey object
    try:
        token_pubkey = Pubkey.from_string(token_address)
    except:
        return {"error": "Invalid token address"}
    
    # STEP 3: Get token's mint account info
    mint_account = solana_client.get_account_info(token_pubkey)
    if not mint_account.value:
        return {"error": "Token account not found"}
    
    mint_data = mint_account.value
    data_bytes = mint_data.data
    
    # STEP 4: Check Mint/Freeze Authorities (critical!)
    # SPL Token data structure offsets:
    # https://spl.solana.com/token#account-structures
    mint_authority_enabled = data_bytes[0] != 0
    mint_authority = None
    if mint_authority_enabled and len(data_bytes) >= 36:
        mint_authority = str(Pubkey.from_bytes(data_bytes[4:36]))
    
    freeze_authority_enabled = False
    freeze_authority = None
    if len(data_bytes) >= 65 and data_bytes[64] != 0:
        freeze_authority_enabled = True
        if len(data_bytes) >= 100:
            try:
                freeze_authority = str(Pubkey.from_bytes(data_bytes[68:100]))
            except Exception as e:
                print(f"Error parsing freeze authority: {e}")
    
    # STEP 5: Check if program is upgradable
    program_id = mint_data.owner
    program_info = solana_client.get_account_info(program_id)
    is_upgradable = program_info.value.owner != Pubkey.from_string("BPFLoaderUpgradeab1e11111111111111111111111")
    
    return {
        "mint_address": token_address,
        "owner": str(mint_data.owner),
        "mint_authority_enabled": mint_authority_enabled,
        "mint_authority": mint_authority,
        "freeze_authority_enabled": freeze_authority_enabled,
        "freeze_authority": freeze_authority,
        "is_upgradable_program": is_upgradable
    }

# Example usage
token_address = "5oVNBeEEQvYi1cX3ir8Dx5n1P7pdxydbGF2X4TxVusJm" # mint enabled
# token_address = "Gx23NkhXZRJauSpEdLcYVDygqHD5SJ6AYEM2AWuYErDz" # upgrade enabled
# token_address = "HRg47EKMZXXUNcFgwUbnYyihjfMSYnoyFn9KNDJrYk97" # freeze enabled
# token_address = "C7heQqfNzdMbUFQwcHkL9FvdwsFsDRBnfwZDDyWYCLTZ" # good one
security_report = check_solana_token_security(token_address)
print(security_report)

{'mint_address': '5oVNBeEEQvYi1cX3ir8Dx5n1P7pdxydbGF2X4TxVusJm', 'owner': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA', 'mint_authority_enabled': True, 'mint_authority': 'AYhux5gJzCoeoc1PoJ1VxwPDe22RwcvpHviLDD1oCGvW', 'freeze_authority_enabled': False, 'freeze_authority': None, 'is_upgradable_program': True}


**Finalized version** - some inaccuracies were found in parsing the data bytes. 

In [43]:
from solders.pubkey import Pubkey
from solana.rpc.api import Client
import base58

def check_solana_token_security(token_address: str):
    """Analyzes a Solana token's smart contract for security risks"""
    
    # STEP 1: Connect to Solana network
    solana_client = Client("https://api.mainnet-beta.solana.com")
    
    # STEP 2: Convert string address to Pubkey object
    try:
        token_pubkey = Pubkey.from_string(token_address)
    except:
        return {"error": "Invalid token address"}
    
    # STEP 3: Get token's mint account info
    mint_account = solana_client.get_account_info(token_pubkey)
    if not mint_account.value:
        return {"error": "Token account not found"}
    
    mint_data = mint_account.value
    # Remove base58 decoding - data is already binary
    data_bytes = mint_data.data
    
    # STEP 4: Check Mint/Freeze Authorities (critical!)
    # SPL Token data structure offsets:
    # https://spl.solana.com/token#account-structures
    mint_authority_enabled = data_bytes[0] != 0
    mint_authority = None
    if mint_authority_enabled and len(data_bytes) >= 36:
        mint_authority = str(Pubkey.from_bytes(data_bytes[4:36]))
    
    freeze_authority_enabled = False
    freeze_authority = None
    if len(data_bytes) >= 82 and data_bytes[46] != 0:
        freeze_authority_enabled = True
        if len(data_bytes) >= 100:
            freeze_authority = str(Pubkey.from_bytes(data_bytes[47:79]))
    
    # STEP 5: Check if program is upgradable
    program_id = mint_data.owner
    program_info = solana_client.get_account_info(program_id)
    is_upgradable = program_info.value.owner == Pubkey.from_string("BPFLoaderUpgradeab1e11111111111111111111111")
    
    return {
        "mint_address": token_address,
        "owner": str(mint_data.owner),
        "mint_authority_enabled": mint_authority_enabled,
        "mint_authority": mint_authority,
        "freeze_authority_enabled": freeze_authority_enabled,
        "freeze_authority": freeze_authority,
        "is_upgradable_program": is_upgradable
    }

# Example usage
# token_address = "5oVNBeEEQvYi1cX3ir8Dx5n1P7pdxydbGF2X4TxVusJm" # mint enabled
# token_address = "Gx23NkhXZRJauSpEdLcYVDygqHD5SJ6AYEM2AWuYErDz" # upgrade enabled
# token_address = "HRg47EKMZXXUNcFgwUbnYyihjfMSYnoyFn9KNDJrYk97" # freeze+mint enabled
token_address = "C7heQqfNzdMbUFQwcHkL9FvdwsFsDRBnfwZDDyWYCLTZ" # good one
security_report = check_solana_token_security(token_address)
print(security_report)

{'mint_address': 'C7heQqfNzdMbUFQwcHkL9FvdwsFsDRBnfwZDDyWYCLTZ', 'owner': 'TokenkegQfeZyiNwAJbNbGKPFXCWuBvf9Ss623VQ5DA', 'mint_authority_enabled': False, 'mint_authority': None, 'freeze_authority_enabled': False, 'freeze_authority': None, 'is_upgradable_program': False}
